In [1]:
import pandas as pd

This notebook will clean data from the California Department of Education for the analysis. Our goal is to merge all the data into a single cohesive geodataframe, which will happen in a subsequent notebook.  

 - lcff data
 - grades
 - demographics/shapefiles

# LCFF data
importing from [California Department of Education](https://ias.cde.ca.gov/lcffsnapshot/lcff.aspx) and cleaning.

In [2]:
lcff = pd.read_csv('lcff_data/lcffsnapshot18an.csv')
# import lcff data from source

In [3]:
lcff.columns = lcff.columns.str.strip()
# strip leading/trailing spaces

In [4]:
lcff.drop(lcff.tail(1).index,inplace=True)
# extracting state totals, removing from df

In [5]:
lcff['County Code'] = lcff['County Code'].astype(str)
lcff['District Code'] = lcff['District Code'].astype(int)
lcff['District Code'] = lcff['District Code'].astype(str)
lcff['School Code'] = lcff['School Code'].astype(int)
lcff['School Code'] = lcff['School Code'].astype(str)
# changing to strings
# slicing by district level cds code

In [6]:
lcff['School Code'] = lcff['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# front fill school codes to == 7

In [7]:
lcff = lcff.loc[lcff['School Code'] == '0000000']
# select districts, district entries do not have a school level cds code

In [8]:
lcff['cds'] = lcff['County Code'] + lcff['District Code'] + lcff['School Code']
# combining into full cds code
col_name='cds'
first_col = lcff.pop(col_name)
lcff.insert(0, col_name, first_col)
# moving to front
lcff.drop('County Code', axis=1, inplace=True)
lcff.drop('School Code', axis=1, inplace=True)
# dropping excess

In [9]:
lcff = lcff.rename(columns={'Charter Number': 'charter_number',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_percentage',
                     'Base Grant Funding\nTarget, C-5': 'base_grant',
                     'Supplemental Grant Funding\nTarget, D-7': 'supplemental_grant',
                     'Concentration Grant Funding\nTarget, E-8': 'concentration_grant',
                     'Total LCFF Target Entitlement\nTarget, G-1/F-1': 'total_grants',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_count',
                     'Necessary Small Schools Allowance\nTarget, F-1': 'nec_small_schools',
                     'Local Educational Agency' : 'districtname'      
                           })
# renaming for readability
lcff = lcff[[  'districtname',
               'unduplicated_pupil_count', 
               'base_grant', 
               'supplemental_grant', 
               'concentration_grant', 
               'total_grants', 
               'cds' ]].copy()
# copying relevant columns into a new dataframe

# Grade data 
importing from [California Department of Education](https://www.cde.ca.gov/ta/ac/cm/datafilesfall18.asp) and cleaning

Math first

In [10]:
math = pd.read_csv('lcff_data/mathdownload2018.csv')

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
replacements = dict(
    studentgroup = {'ALL':'all_students',
                        'AA':'african_american',
                        'AI':'american_indian',
                        'AS':'asian',
                        'FI':'filipino',
                        'HI':'hispanic',
                        'PI':'pacific_islander',
                        'WH':'white',
                        'MR':'multiple_races',
                        'EL':'english_learners',
                        'ELO':'english_learners_only',
                        'RFP':'rfep_only',
                        'EO':'english_only',
                        'SED':'ses_disad',
                        'SWD':'disabilities',
                        'FOS':'foster_youth',
                        'HOM':'homeless_youth',
                })
# dictionary for replacing student group codes

In [12]:
math['studentgroup'].astype(str)
math = math.replace(replacements)
# replace student group codes

fixing cds codes to match to enable merges

In [13]:
math['cds'].astype(str)
# changing to string
math = math[math.coe_flag !='Y']
# removing county offices of education
math = math.loc[math['rtype'] == 'D']
# select district-level performance
math['cds'] = math['cds'].apply(lambda x: '{0:0>14}'.format(x))
# adding leading zero to 13 digit int
math['District Code'] = [x[2:7] for x in math['cds']]
# adding district only code

In [14]:
math = pd.concat([math.pop(x) for x in ['District Code',
                                          'districtname',                                          
                                          'studentgroup', 
                                          'currstatus', 
                                          'priorstatus', 
                                          'change',
                                          'countyname']],
                    1)
# slice out columns

In [15]:
math.to_csv("math.csv")
# save to disk

English grades now

In [16]:
ela = pd.read_csv('lcff_data/eladownload2018.csv')

In [17]:
ela['studentgroup'].astype(str)
ela = ela.replace(replacements)
# replace student group codes

fixing cds codes to match to enable merges

In [18]:
ela['cds'].astype(str)
# changing cds code to string
ela = ela[ela.coe_flag !='Y']
# removing county offices of education
ela = ela.loc[ela['rtype'] == 'D']
# select district level entries
ela['cds'] = ela['cds'].apply(lambda x: '{0:0>14}'.format(x))
# adding leading zero to 13 digit int
ela['District Code'] = [x[2:7] for x in ela['cds']]

In [20]:
#creating district-level ela performance variable (dictionary)
ela = pd.concat([ela.pop(x) for x in ['District Code',
                                          'districtname',                                          
                                          'studentgroup', 
                                          'currstatus', 
                                          'priorstatus', 
                                          'change',
                                          'countyname']],
                    1)
# slice out columns

In [21]:
ela.to_csv("ela.csv")
# save to disk

# Student attribute and shapefile data

this requires geopandas

In [23]:
import geopandas as gpd

In [24]:
t = gpd.read_file("California_School_District_Areas_2018-19-shp/DistrictAreas1819.shp")

In [25]:
t.head()

,OBJECTID,Year,FedID,CDCode,CDSCode,CountyName,DistrictNa,DistrictTy,GradeLow,GradeHigh,...,AbsentPct,GradCohort,GradPct,UCCSUReqMe,DropOutPct,EnrollCumu,SuspPct,SHAPE_Leng,SHAPE_Area,geometry
0,1,2018-19,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,KG,12,...,9.4,882.0,88.900002,58.900002,5.2,11600.0,2.6,57196.403944,4.733304e+07,"MULTIPOLYGON (((-13606221.352 4540862.002, -13..."
1,2,2018-19,0601860,0161127,01611270000000,Alameda,Albany City Unified,Unified,KG,12,...,6.4,301.0,94.699997,63.900002,1.7,3830.0,1.9,12870.772156,7.161276e+06,"POLYGON ((-13612889.056 4565138.930, -13612894..."
2,3,2018-19,0604740,0161143,01611430000000,Alameda,Berkeley Unified,Unified,KG,12,...,9.7,856.0,87.400002,83.300003,10.7,10534.0,1.4,44426.287405,4.376389e+07,"POLYGON ((-13609481.010 4565073.896, -13609481..."
3,4,2018-19,0607800,0161150,01611500000000,Alameda,Castro Valley Unified,Unified,KG,12,...,7.4,742.0,92.599998,55.900002,3.0,9532.0,2.2,142338.494091,2.833205e+08,"MULTIPOLYGON (((-13581537.591 4528171.452, -13..."
4,5,2018-19,0612630,0161168,01611680000000,Alameda,Emery Unified,Unified,KG,12,...,20.5,40.0,100.000000,40.000000,0.0,780.0,3.6,13762.869292,5.368212e+06,"POLYGON ((-13614035.309 4555653.898, -13614035..."


In [26]:
t.to_file("attributes.shp")
# save to disk